## Шаг 1: импорт библиотек и мат. констант

In [223]:
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import interpolate, signal #, stats
from math import e, pi
from numpy import fft
import numpy as np
import pandas as pd
import os.path

## Шаг  2: загрузка всех файлов и указание входных данных

In [229]:
# загрузка всех файлов и входные данные

start_point = 0 # начальная точка спектра (для устранения нулей там, где спектра нет)
finish_point = 512 # конечная точка спектра
focal_length = 3000 # фокусное расстояние объектива
num_of_steps_first_galva = 1 # количество шагов первого гальвы, то есть количество файлов для обработки
phase_iterations = 150
num_of_steps_second_galva = 1 # количество шагов второго гальвы
start_1g = 49 # начало фазы зеркала для 1 гальвы (если скан образца сделан с тем же шагом, но в меньшем диапазоне, чем зеркало)
start_2g = 49 # начало фазы зеркала для 2 гальвы
translator_step = '51355'
translator_step_ref = '50355'
name_of_folder_to_load = 'ph_bead60' # название папки для загрузки данных (спектр образца)
name_of_folder_to_load_mirrorphase = 'Mirrorphase_512' # название папки для загрузки данных (фаза зеркала)
name_of_folder_to_save = 'Phase results ph_bead60' # название папки для сохранения результатов

dir_to_load = 'D:/Data/OCT/2019_07_06' # директория для загрузки папки
path_of_folder_to_load = os.path.join(dir_to_load, name_of_folder_to_load + '/')
path_of_folder_to_load_sample = os.path.join(dir_to_load, name_of_folder_to_load + '/' + translator_step + '/')
path_of_folder_to_load_ref = os.path.join(dir_to_load, name_of_folder_to_load + '/' + translator_step_ref + '/')

dir_to_load_mirrorphase = 'D:/Data/OCT/2018_07_28/mirror_50x50_101steps_gold_nearest' # директория для загрузки папки
path_of_folder_to_load_mirrorphase = os.path.join(dir_to_load_mirrorphase, name_of_folder_to_load_mirrorphase + '/')

wavelength = pd.read_csv(path_of_folder_to_load + '/' + 'Wavelength.txt', header = None, sep = '\t') # загрузка файла длин волн
Lateral_y = pd.read_csv(path_of_folder_to_load + '/' + 'Lateral_y.txt', header = None, sep = '\t')#загрузка файла напряжений 1 гальвы
#Lateral_x = pd.read_csv(path_of_folder_to_load + 'Lateral_x.txt', header = None, sep = '\t')#загрузка файла напряжений 2 гальвы

dir_to_save = 'D:/Data/OCT/2019_07_06' # директория для сохранения папки
path_of_folder_to_save = os.path.join(dir_to_save, name_of_folder_to_save + '.h5')
path_of_folder_to_save1 = os.path.join(dir_to_save, name_of_folder_to_save + 'abs.h5')

#if os.path.exists(path_of_folder_to_save):
#    print 'Папка уже существует!'
#else:
    #os.makedirs(path_of_folder_to_save) # создание папки, если не существует

## Шаг 3: применение фазы Гильберта 
Желательно вырезать из спектра нули слева и справа перед этим этапом!
Массивы должны быть одного размера.

In [230]:
galva_1_step = 0
phase_num = np.arange(0, phase_iterations, 1)
df=pd.DataFrame(index=['Galva 1 ↓'] + np.array(Lateral_y.iloc[:,0]).tolist(), 
                columns=['Depth ↓ \\ Phase Num →'] + phase_num.tolist())

uniform_k = (2*pi*1E9*(np.arange(0.,len(wavelength.index),1.)/(len(wavelength.index)-1) # переход к равном. k
                    *float(1./wavelength.min(axis=0)-1./wavelength.max(axis=0)) 
                       + float(1./wavelength.max(axis=0)))).reshape(512,1)

nonuniform_wavelength = (2*pi*1E9/np.flipud(uniform_k)).reshape(512,1) # переход к неравном длинам волн

# микроны по глубине
df.iloc[0,0] = 0.5*2*pi*1E6*(np.arange(0., (finish_point-start_point), 1.))/(uniform_k[len(uniform_k)
                                                                                         -finish_point:len(uniform_k)
                                                                                         -start_point].max() 
                                                                               - uniform_k[len(uniform_k)
                                                                                           -finish_point:len(uniform_k)
                                                                                           -start_point].min()) 
df_abs = pd.DataFrame(index=['Galva 1 ↓'] + np.array(Lateral_y.iloc[:,0]).tolist(), 
                columns=['Depth ↓ \\ Phase Num →'] + phase_num.tolist())
df_abs.iloc[0,0] = df.iloc[0,0]

In [231]:
while galva_1_step < num_of_steps_first_galva:
    
    # загрузка файлов 
    sig_input = pd.read_csv(path_of_folder_to_load_sample + str(galva_1_step) + '.txt', header = None, sep = '\t') 
    # загрузка файла спектра зеркала
    sig_no_interference = pd.read_csv(path_of_folder_to_load_ref + str(galva_1_step) + '.txt', header = None, sep = '\t') 
    # загрузка файла спектра без интерференции
    
    mirror_amp_phase = pd.read_csv(path_of_folder_to_load_mirrorphase + str(galva_1_step + start_1g) 
                                        + '.txt', header = None, sep = '\t') 
    # загрузка файла фазы зеркала
    
    if len(sig_no_interference.columns) == 1:
        sig_no_interference1 = pd.DataFrame(index = sig_input.index, columns = sig_input.columns)
        k = 0
        while k < len(sig_input.columns):
            sig_no_interference1[[k]] = sig_no_interference[[0]]
            k+=1
        sig_no_interference = sig_no_interference1
        
    #устранение пика от пинцента    
    sig_input[:][80:103] = 1.
    sig_no_interference[:][80:103] = 1.
    
    # первичная обработка сигнала
    sig_norm = (sig_input - sig_no_interference)/sig_no_interference # нормировка сигнала на спектр без интерференции

    uniform_k = (2*pi*1E9*(np.arange(0.,len(wavelength.index),1.)/(len(wavelength.index)-1) # переход к равном. k
                    *float(1./wavelength.min(axis=0)-1./wavelength.max(axis=0)) 
                       + float(1./wavelength.max(axis=0)))).reshape(512,1)

    nonuniform_wavelength = (2*pi*1E9/np.flipud(uniform_k)).reshape(512,1) # переход к неравном длинам волн

    # интерполяция в неравномерных длинах волн
    signal_k = pd.DataFrame(index=range(0,finish_point-start_point), columns=range(0,len(sig_norm.columns)+1))
    signal_k[[0]] = uniform_k[len(uniform_k)-finish_point:len(uniform_k)-start_point]

    k=0
    while k < len(sig_norm.columns): # применение интерполяции для каждого столбца (положения 2 гальвазеркала)
        tck = interpolate.splrep(wavelength[[0]], sig_norm[[k]], s=0)
        interpolated = interpolate.splev(nonuniform_wavelength.reshape(512,1), tck, der=0) 
        signal_k[[k+1]] = np.flipud(interpolated[start_point:finish_point])
        k += 1

    sample_no_const = signal_k # если пропускается шаг 4 — активировать

    k=1
    while k < len(sample_no_const.columns): # применение преобразований к каждому положению 2 гальвазеркала
        df.iloc[galva_1_step + 1,k] = np.angle(fft.ifft(sample_no_const.iloc[:,k]
                                                      *e**(1j*mirror_amp_phase.iloc[start_point:finish_point,1 + start_2g])))
        df_abs.iloc[galva_1_step+1,k] = np.abs(fft.ifft(sample_no_const.iloc[:,k]
                                                      *e**(1j*mirror_amp_phase.iloc[start_point:finish_point,1 + start_2g])))
        # применениe ТОЛЬКО фазы зеркала к спектру образца
        k += 1    
    
    galva_1_step += 1

In [232]:
df # смотрим, как выглядит документ. Всё ок

,Depth ↓ \ Phase Num →,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
Galva 1 ↓,"[0.0, 0.9534853904434702, 1.9069707808869405, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0.0,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [228]:
# сохранение файла в формате hdf
df.to_hdf(path_of_folder_to_save, key='phase', mode = 'w')
df_abs.to_hdf(path_of_folder_to_save1, key='image', mode = 'w')